In [3]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine

load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

DB_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine_entrada = create_engine(DB_URL)


query = "SELECT * FROM Analista"


dados_originais_df = pd.read_sql(query, engine_entrada)


dados_originais_df = dados_originais_df.copy()


In [4]:
# pseudonimizar nome

def pseudonimizar_nome(nome):

    for i in range (len(nome)):
        nome= str("Analista "+str(i))
        return nome
dados_anonimizados = dados_originais_df.copy()
dados_anonimizados["nome_pseudonimizado"] = [
    f"Analista {i+1}" for i in range(len(dados_anonimizados))
]

print("==== Dados anonimizados ====")
display(dados_anonimizados.loc[:, ["nome_pseudonimizado"]])
print("\n==== Dados originais preservados ====")
display(dados_anonimizados.loc[:, ["nome_completo"]])

==== Dados anonimizados ====


,nome_pseudonimizado
0,Analista 1
1,Analista 2
2,Analista 3
3,Analista 4
4,Analista 5
5,Analista 6
6,Analista 7
7,Analista 8
8,Analista 9
9,Analista 10



==== Dados originais preservados ====


,nome_completo
0,Rafael Henrique Silva
1,Camila Rodrigues Lima
2,Leonardo Santos Pereira
3,Fernanda Oliveira Costa
4,Gabriel Alves Souza
5,Amanda Torres Dias
6,Thiago Barbosa Nunes
7,Larissa Martins Rocha
8,Felipe Cardoso Teixeira
9,Natália Ferreira Santos


In [12]:
# cripto senha

import random
import string

def cifra_cesar(texto):
   
    chave = random.randint(1, 25)
    resultado = ""

    
    letras = list(string.ascii_uppercase) 
    simbolos = ['@', '#', '$', '%', '&', '*', '!', '?', '+', '=']

    for char in str(texto):
        if char.isalpha():  
            base = ord('A') if char.isupper() else ord('a')
            resultado += chr((ord(char) - base + chave) % 26 + base)
        elif char.isdigit():  #letras ou símbolos aleatórios
            if random.choice([True, False]):
                resultado += random.choice(letras)
            else:
                resultado += random.choice(simbolos)
        else:
            resultado += char  

    return resultado


dados_anonimizados["senha_pseudonimizada"] = dados_anonimizados["senha"].apply(cifra_cesar)

print("==== Dados anonimizados ====")
display(dados_anonimizados.loc[:, ["senha_pseudonimizada"]])

print("\n==== Dados originais preservados ====")
display(dados_anonimizados.loc[:, ["senha"]])



==== Dados anonimizados ====


,senha_pseudonimizada
0,KROYC&@Q=
1,bmeeiadpI%$
2,msanpu=XU
3,&@*WCL+H
4,firns#K@
5,qclfyJM$
6,nop?CXR%
7,YO*$#TIM
8,cnffjbeq
9,D?#QE@@ZA



==== Dados originais preservados ====


,senha
0,123456789
1,password123
2,qwerty123
3,12345678
4,admin123
5,senha123
6,abc12345
7,11111111
8,password
9,123123123


In [6]:
from IPython.display import display

# Pseudonimização da data de contratação
def pseudonimizar_data(dt_contratacao):
    # Garante string e remove espaços
    data = str(dt_contratacao).strip()
    
    # Remove hífen e barra
    data = data.replace("-", "").replace("/", "")
    
    # Mostra só os 4 primeiros caracteres (ano) + ****
    if len(data) > 4:
        return data[:4] + "****"
    else:
        return data + "****"

# Aplica pseudonimização
dados_anonimizados["data_contratacao_pseudo"] = dados_anonimizados["dt_contratacao"].apply(pseudonimizar_data)

print("==== Dados anonimizados ====")
display(dados_anonimizados.loc[:, ["dt_contratacao", "data_contratacao_pseudo"]])


==== Dados anonimizados ====


,dt_contratacao,data_contratacao_pseudo
0,2023-01-15,2023****
1,2023-02-20,2023****
2,2023-03-10,2023****
3,2023-04-05,2023****
4,2023-05-12,2023****
5,2023-06-18,2023****
6,2023-07-22,2023****
7,2023-08-30,2023****
8,2023-09-15,2023****
9,2023-10-08,2023****


In [7]:
from IPython.display import display

def pseudonimizar(cargo):
    # Garante string
    palavra = str(cargo).strip()
    
    # Remove hífens e espaços
    palavra = palavra.replace("-", "").replace(" ", "")
    
    # Substitui a partir do 5º caractere por ****
    if len(palavra) > 3:
        valor_pseudo = palavra[:3] + "****"
    else:
        valor_pseudo = palavra + "****"
    
    return valor_pseudo

# Cria cópia do DataFrame original (opcional, pra não alterar os dados reais)
dados_anonimizados = dados_originais_df.copy()

# Aplica pseudonimização na coluna "cargo"
dados_anonimizados["cargo_pseudonimizado"] = dados_anonimizados["cargo"].apply(pseudonimizar)

print("==== Dados anonimizados ====")
display(dados_anonimizados.loc[:, ["cargo_pseudonimizado"]])

print("\n==== Dados originais preservados ====")
display(dados_anonimizados.loc[:, ["cargo"]])



==== Dados anonimizados ====


,cargo_pseudonimizado
0,Ana****
1,Ana****
2,Ana****
3,Coo****
4,Ana****
5,Ana****
6,Ana****
7,Ana****
8,Sup****
9,Ana****



==== Dados originais preservados ====


,cargo
0,Analista Sênior
1,Analista Pleno
2,Analista Junior
3,Coordenador de Análises
4,Analista Sênior
5,Analista de Operações
6,Analista de Logística
7,Analista de Transporte
8,Supervisor de Análises
9,Analista Executivo


In [8]:
# Anonimização analista 

from IPython.display import display


def anonimizar(senha, chave):
    senha = str(senha)
    chave = str(chave).lower()
    senha_cripto = ''
    i = 0
    for l in senha:
        deslocamento = ord(chave[i % (len(chave))]) - ord('a')
        nova_letra = chr((ord(l) - 32 + deslocamento) % 95 + 32)
        senha_cripto += nova_letra
        i += 1
    return senha_cripto

# cópia
dados_anonimizados = dados_originais_df.copy()


chave = "minhachave"
dados_anonimizados["cpf_anonimizado"] = dados_anonimizados["cpf"].apply(lambda x: anonimizar(x, chave))

print("==== Dados anonimizados ====")
display(dados_anonimizados.loc[:, ["cpf_anonimizado"]])

print("\n==== Dados originais preservados ====")
display(dados_anonimizados.loc[:, ["cpf"]])

==== Dados anonimizados ====


,cpf_anonimizado
0,=:@;58>8N5>
1,>;A<69?9E6?
2,?<B=7:@0F7@
3,@=C>8;71G8A
4,A>D?9282H9B
5,B?E@0393I:C
6,C@F714:4J;D
7,DA=825;5K<E
8,E8>936<6L==
9,<9?:47=7M=>



==== Dados originais preservados ====


,cpf
0,12345678912
1,23456789023
2,34567890134
3,45678901245
4,56789012356
5,67890123467
6,78901234578
7,89012345689
8,90123456791
9,01234567892


In [9]:
# Pseudonimização email
def pseudonimizar(email):
    email = str(email)
    parte_local, dominio = email.split("@")
  
    if len(parte_local) > 3:
        parte_local_pseudo = parte_local[:2] + "****"
    else:
        parte_local_pseudo = parte_local + "****"
    return parte_local_pseudo + "@" + dominio
dados_anonimizados["email_pseudonimizado"] = dados_anonimizados["email"].apply(pseudonimizar)

print("==== Dados anonimizados ====")
display(dados_anonimizados.loc[:, ["email_pseudonimizado"]])

print("\n==== Dados originais preservados ====")
display(dados_anonimizados.loc[:, ["email"]])

==== Dados anonimizados ====


,email_pseudonimizado
0,ra****@eitruck.com
1,ca****@eitruck.com
2,le****@eitruck.com
3,fe****@eitruck.com
4,ga****@eitruck.com
5,am****@eitruck.com
6,th****@eitruck.com
7,la****@eitruck.com
8,fe****@eitruck.com
9,na****@eitruck.com



==== Dados originais preservados ====


,email
0,rafael.silva@eitruck.com
1,camila.lima@eitruck.com
2,leonardo.pereira@eitruck.com
3,fernanda.costa@eitruck.com
4,gabriel.souza@eitruck.com
5,amanda.dias@eitruck.com
6,thiago.nunes@eitruck.com
7,larissa.rocha@eitruck.com
8,felipe.teixeira@eitruck.com
9,natalia.santos@eitruck.com
